In [1]:
import numpy as np
import pandas as pd

In [2]:
def chi_square(df,attribute,target):
    attribute_values=df[attribute].unique()
    target_values=df[target].unique()
    chi=0
    for attr_value in attribute_values:
        for targ_value in target_values:
            observed_freq=len(df[(df[attribute]==attr_value)&(df[target]==targ_value)])
            attr_prob=len(df[df[attribute]==attr_value])
            targ_prob=len(df[df[target]==targ_value])
            expected_freq=(attr_prob*targ_prob)/len(df)
            chi+=(observed_freq-expected_freq)**2/expected_freq
    return chi

In [48]:
df=pd.read_csv(r"C:\Users\jasme\Downloads\weather.csv")

In [8]:
chi_square(df,'Humidity','Play')

2.8

In [18]:
def chi_square_selection(df,target,k):
    col_names=df.columns
    chi_values={}
    for col in col_names:
        if col==target:
            continue
        chi_values[col]=chi_square(df,col,target)
    sorted_features=sorted(chi_values.items(),key=lambda x: x[1], reverse=True)
    selected_features=[f for f,_ in sorted_features[:k]]
    return selected_features
        

In [19]:
chi_square_selection(df,'Play',2)

['Outlook', 'Humidity']

In [59]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Outlook']=le.fit_transform(df['Outlook'])
df['Temp']=le.fit_transform(df['Temp'])
df['Windy']=le.fit_transform(df['Windy'])
df['Humidity']=le.fit_transform(df['Humidity'])
df['Play']=le.fit_transform(df['Play'])


In [30]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
chi=SelectKBest(chi2,k=2)
test=chi.fit(df.iloc[:,:-1],df.iloc[:,-1])
test.scores_

array([2.02814815, 0.11690821, 1.4       , 0.4       ])

In [31]:
df.columns

Index(['Outlook', 'Temp', 'Humidity', 'Windy', 'Play'], dtype='object')

In [38]:
def entropy(df):
    target=df.keys()[-1]
    #print(target)
    target_values=df[target].unique()
    #print(target_values)
    entropy=0
    for value in target_values:
        prob=len(df[df[target]==value])/len(df)
        entropy+=-prob*np.log2(prob)
    return entropy

In [39]:
entropy(df)

0.9402859586706309

In [50]:
def find_average_info_entropy(df,attribute,target):
    attribute_values=df[attribute].unique()
    target_values=df[target].unique()
    avg_info_entropy=0
    for attr_value in attribute_values:
        entropy_subsample=0
        for targ_value in target_values:
            num=len(df[(df[attribute]==attr_value)&(df[target]==targ_value)])
            den=len(df[df[attribute]==attr_value])
            prob=num/den
            entropy_subsample+=-prob*np.log2(prob+1e-7)
        weight=len(df[df[attribute]==attr_value])/len(df)
        avg_info_entropy+=weight*entropy_subsample
    return avg_info_entropy

In [51]:
entropy(df)-find_average_info_entropy(df,'Outlook','Play')

0.2467500670935654

In [52]:
def info_gain_select(df,target,k):
    cols=df.columns
    IG={}
    for col in cols:
        if col==target:
            continue
        IG[col]=entropy(df)-find_average_info_entropy(df,col,target)
    sorted_values=sorted(IG.items(),key=lambda x: x[1],reverse=True)
    sorted_features=[f for f,_ in sorted_values[:k]]
    return sorted_features
    

In [55]:
info_gain_select(df,'Play',2)

['Outlook', 'Humidity']

In [60]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
test=SelectKBest(mutual_info_classif,k=2)
test.fit(df.iloc[:,:-1],df.iloc[:,-1])
test.scores_

array([0.04427775, 0.12783784, 0.199068  , 0.048246  ])

In [61]:
from sklearn import datasets
iris=datasets.load_iris()
X=iris.data
Y=iris.target

In [62]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100)
rf.fit(X_train,Y_train)

RandomForestClassifier()

In [64]:
rf.feature_importances_

array([0.07145637, 0.02981211, 0.47361564, 0.42511588])

In [65]:
from sklearn.feature_selection import SelectFromModel
clf=SelectFromModel(RandomForestClassifier(n_estimators=100))
clf.fit(X,Y)

SelectFromModel(estimator=RandomForestClassifier())

In [70]:
clf.fit_transform(X,Y)

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1],
       [1.5, 0.2],
       [1.6, 0.2],
       [1.4, 0.1],
       [1.1, 0.1],
       [1.2, 0.2],
       [1.5, 0.4],
       [1.3, 0.4],
       [1.4, 0.3],
       [1.7, 0.3],
       [1.5, 0.3],
       [1.7, 0.2],
       [1.5, 0.4],
       [1. , 0.2],
       [1.7, 0.5],
       [1.9, 0.2],
       [1.6, 0.2],
       [1.6, 0.4],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.6, 0.2],
       [1.6, 0.2],
       [1.5, 0.4],
       [1.5, 0.1],
       [1.4, 0.2],
       [1.5, 0.2],
       [1.2, 0.2],
       [1.3, 0.2],
       [1.4, 0.1],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.3, 0.3],
       [1.3, 0.3],
       [1.3, 0.2],
       [1.6, 0.6],
       [1.9, 0.4],
       [1.4, 0.3],
       [1.6, 0.2],
       [1.4, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [4.7, 1.4],
       [4.5, 1.5],
       [4.9,